In [4]:
pip install SpeechRecognition pydub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 32.3 MB/s eta 0:00:00


In [5]:
import speech_recognition as sr
import subprocess
from pydub import AudioSegment
from pydub.silence import split_on_silence
from google.colab import files
import os

In [6]:
uploaded = files.upload()

Saving The Climb audiobook.mp3 to The Climb audiobook.mp3


In [7]:
input_audio_file = list(uploaded.keys())[0]

In [8]:
output_wav_file = "wav_file.wav"
subprocess.call(['ffmpeg', '-i', input_audio_file, output_wav_file])


0

In [9]:
r = sr.Recognizer()

In [10]:
def test_audio_recognition(path):
    """
    Test speech recognition on the provided audio file.
    """
    try:
        with sr.AudioFile(path) as source:
            audio_listened = r.record(source)
            text = r.recognize_google(audio_listened, language="en-US")
            print(f"Recognized text: {text}")
            return text
    except sr.UnknownValueError:
        print("Error: Unable to recognize speech")
        return None
    except sr.RequestError as e:
        print(f"Request Error: {str(e)}")
        return None

def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and applying speech recognition on each of these chunks.
    """
    # Load the audio file using pydub
    sound = AudioSegment.from_wav(path)

    # Split audio sound where silence is 500 ms or more and get chunks
    chunks = split_on_silence(sound,
        min_silence_len = 500,
        silence_thresh = sound.dBFS - 14,
        keep_silence = 500,
    )

    # Directory to store the audio chunks
    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = ""

    # Process each chunk
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")

        # Print chunk details for debugging
        print(f"Processing chunk {i}, length: {len(audio_chunk)} ms")

        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            try:
                text = r.recognize_google(audio_listened, language="en-US")
                print(f"Chunk {i} recognized: {text}")
                whole_text += text.capitalize() + ". "
            except sr.UnknownValueError:
                print(f"Error with chunk {i}: Unable to recognize speech")

    # Return the complete transcription
    return whole_text

In [11]:
print("Testing full audio file:")
print(test_audio_recognition(output_wav_file))


Testing full audio file:
Recognized text: what's the possibility of an attraction or a relationship was off the table they could become friends with none of the usual boy girl pheromones interfering since then they'd never returned to the subject of us as a couple but lately things were becoming weird you felt lighter and full of hope maybe this was finally his chance to make a move ever since he started high school she had been his one and only crush and ask her out of the year 7 K Camp and she turned him down saying she was waiting to have a boyfriend when she was older at first he thought she'd just been making up an excuse to reject him like all the other girls here return that she kept saying no to everyone she wasn't interested in a boyfriend and so he became content to wait he figured it was only a matter of time until she wanted someone naked finally crossed over from Friends into something more all he wanted was the fearer
what's the possibility of an attraction or a relations

In [12]:
print("\nTesting chunked audio file:")
print(get_large_audio_transcription(output_wav_file))


Testing chunked audio file:
Processing chunk 1, length: 4908 ms
Chunk 1 recognized: what's the possibility of an attraction or a relationship was off the table
Processing chunk 2, length: 5383 ms
Chunk 2 recognized: they could become friends with none of the usual boy girl pheromones interfering
Processing chunk 3, length: 4564 ms
Chunk 3 recognized: since then they've never returned to the subject of us as a couple
Processing chunk 4, length: 2775 ms
Chunk 4 recognized: but lately things were becoming weird
Processing chunk 5, length: 1454 ms
Chunk 5 recognized: he felt light
Processing chunk 6, length: 1348 ms
Chunk 6 recognized: and full of heart
Processing chunk 7, length: 3022 ms
Chunk 7 recognized: maybe this was finally his chance to make a move
Processing chunk 8, length: 2260 ms
Chunk 8 recognized: ever since he started High School
Processing chunk 9, length: 2787 ms
Chunk 9 recognized: cheapest one and only Crush
Processing chunk 10, length: 6621 ms
Chunk 10 recognized: Vien